In [114]:
import pandas as pd
import numpy as np
import re

In [109]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

In [94]:
data =pd.read_csv("/content/COVIDSenti-A.csv")

In [3]:
data.head()

,tweet,label
0,Coronavirus | Human Coronavirus Types | CDC ht...,neu
1,"@shehryar_taseer That‚Äôs üíØ true , \nCorona...",neu
2,"TLDR: Not SARS, possibly new coronavirus. Diff...",neg
3,Disease outbreak news from the WHO: Middle Eas...,neu
4,China - Media: WSJ says sources tell them myst...,neu


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   30000 non-null  object
 1   label   30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


In [ ]:
data['label'].value_counts()  #checking for no of unique values

neu    22949
neg     5083
pos     1968
Name: label, dtype: int64

In [5]:
data.columns

Index(['tweet', 'label'], dtype='object')

creating a function to remove unncessary values and alphanumerics in the doc
removed @ tag,numbers, urls , hash tags and other symbols


In [95]:
documents_clean = []
for v in data["tweet"]:
    tweet = re.sub('[^\x00-\x7F]+', ' ', v)
    tweet = re.sub('@\w+', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub('[0-9]', '', tweet)
    tweet = re.sub('\s{2,}', ' ', tweet)
    tweet=re.sub('http\S+', '', tweet)
    tweet=re.sub('[^a-zA-z#]+', ' ', tweet)
    tweet=tweet.replace('#','')
    documents_clean.append(tweet)

splitting tweets and labels

In [96]:
X=documents_clean

In [88]:
# tweet=pd.DataFrame(X) 

In [89]:
# X = tweet

In [71]:
X


,0
0,coronavirus human coronavirus types cdc
1,that s true corona virus swine flue bird flu ...
2,tldr not sars possibly new coronavirus difficu...
3,disease outbreak news from the who middle east...
4,china media wsj says sources tell them mystery...
...,...
29995,cdc re test confirms westerdam cruise ship pas...
29996,two doctors die of coronavirus within hours in...
29997,beijing the lockdown of guo jing s neighbourho...
29998,coronavirusoutbreak in balochistan cpec a rout...


In [97]:
y=data.drop('tweet',axis=1)

In [98]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=101)

text are converted into vectors using TFIDF

In [99]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [100]:
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

using support vector classifier from SVM

In [101]:
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_vectors, y_train)
prediction_linear = classifier_linear.predict(test_vectors)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [112]:
report = classification_report(y_test, prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])
print('neutral: ', report['neu'])

positive:  {'precision': 0.8171806167400881, 'recall': 0.6042345276872965, 'f1-score': 0.6947565543071161, 'support': 614}
negative:  {'precision': 0.8772694262890341, 'recall': 0.7788523533204385, 'f1-score': 0.825136612021858, 'support': 1551}
neutral:  {'precision': 0.919514576649463, 'recall': 0.9644476956839795, 'f1-score': 0.9414453013424736, 'support': 6835}


Got an accuracy of 81 for positive and 87 for negative

giving custom input to check if it positive, negative or neutral

In [108]:
T1="Covid made a terrible future"
review_vector2 = vectorizer.transform([T1]) 
print(classifier_linear.predict(review_vector2))

['neg']


In [105]:
T2="Covid is a disaster"
review_vector2 = vectorizer.transform([T2]) 
print(classifier_linear.predict(review_vector2))

['neu']
